<a href="https://colab.research.google.com/github/anhle/AI_for_Medicine_Specification/blob/master/AI_treatment/week3/AI4M_C3_M3_lecture_notebook_gradcam_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GradCAM: Continuation (Part 2) - Lecture Notebook

In the previous lecture notebook (GradCAM Part 1) we explored what Grad-Cam is and why it is useful. We also looked at how we can compute the activations of a particular layer using Keras API. In this notebook we will check the other element that Grad-CAM requires, the gradients of the model's output with respect to our desired layer's output. This is the "Grad" portion of Grad-CAM. 

Let's dive into it!

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#unzip data to '/content' folder
!unzip '/content/drive/My Drive/data/AI_for_Medicine_Specification/AI_diagnosis/w1/small_data.zip'
!cp '/content/drive/My Drive/data/AI_for_Medicine_Specification/AI_diagnosis/w1/valid-small.csv' /content/
!cp '/content/drive/My Drive/data/AI_for_Medicine_Specification/AI_diagnosis/w1/train-small.csv' /content/
!cp '/content/drive/My Drive/data/AI_for_Medicine_Specification/AI_diagnosis/w1/test.csv' /content/
!cp '/content/drive/My Drive/data/AI_for_Medicine_Specification/AI_diagnosis/w1/densenet.hdf5' /content/
!cp '/content/drive/My Drive/data/AI_for_Medicine_Specification/AI_diagnosis/w1/pretrained_model.h5' /content/

In [0]:
pip install lifelines

In [0]:
#UTILS FILE
import keras
from keras.applications.densenet import DenseNet121
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping, TensorBoard
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import cv2
import pickle



# For part 2
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

import lifelines

IMAGE_DIR = "small_data/"

def get_mean_std_per_batch(df, H=320, W=320):
    sample_data = []
    for idx, img in enumerate(df.sample(100)["Image"].values):
        path = IMAGE_DIR + img
        sample_data.append(np.array(image.load_img(path, target_size=(H, W))))

    mean = np.mean(sample_data[0])
    std = np.std(sample_data[0])
    return mean, std    

def load_image_normalize(path, mean, std, H=320, W=320):
    x = image.load_img(path, target_size=(H, W))
    x -= mean
    x /= std
    x = np.expand_dims(x, axis=0)
    return x

def load_image(path, df, preprocess=True, H = 320, W = 320):
    """Load and preprocess image."""
    x = image.load_img(path, target_size=(H, W))
    if preprocess:
        mean, std = get_mean_std_per_batch(df, H=H, W=W)
        x -= mean
        x /= std
        x = np.expand_dims(x, axis=0)
    return x

def compute_gradcam(model, img, data_dir, df, labels, selected_labels, layer_name='bn'):
    img_path = data_dir + img
    preprocessed_input = load_image(img_path, df)
    predictions = model.predict(preprocessed_input)
    print("Ground Truth: ", ", ".join(np.take(labels, np.nonzero(df[df["Image"] == img][labels].values[0]))[0]))

    plt.figure(figsize=(15, 10))
    plt.subplot(151)
    plt.title("Original")
    plt.axis('off')
    plt.imshow(load_image(img_path, df, preprocess=False), cmap='gray')
    
    j = 1
    for i in range(len(labels)):
        if labels[i] in selected_labels:
            print("Generating gradcam for class %s (p=%2.2f)" % (labels[i], round(predictions[0][i], 3)))
            gradcam = grad_cam(model, preprocessed_input, i, layer_name)
            plt.subplot(151 + j)
            plt.title(labels[i] + ": " + str(round(predictions[0][i], 3)))
            plt.axis('off')
            plt.imshow(load_image(img_path, df, preprocess=False), cmap='gray')
            plt.imshow(gradcam, cmap='jet', alpha=min(0.5, predictions[0][i]))
            j +=1


def cindex(y_true, scores):
    return lifelines.utils.concordance_index(y_true, scores)

# LOAD MODEL FROM C1M2
def load_C3M3_model():
    labels = ['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis',
              'Pneumothorax', 'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation']

    train_df = pd.read_csv("train-small.csv")
    valid_df = pd.read_csv("valid-small.csv")
    test_df = pd.read_csv("test.csv")

    class_pos = train_df.loc[:, labels].sum(axis=0)
    class_neg = len(train_df) - class_pos
    class_total = class_pos + class_neg

    pos_weights = class_pos / class_total
    neg_weights = class_neg / class_total
    print("Got loss weights")
    # create the base pre-trained model
    base_model = DenseNet121(weights='densenet.hdf5', include_top=False)
    print("Loaded DenseNet")
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # and a logistic layer
    predictions = Dense(len(labels), activation="sigmoid")(x)
    print("Added layers")

    model = Model(inputs=base_model.input, outputs=predictions)

    def get_weighted_loss(neg_weights, pos_weights, epsilon=1e-7):
        def weighted_loss(y_true, y_pred):
            # L(X, y) = −w * y log p(Y = 1|X) − w *  (1 − y) log p(Y = 0|X)
            # from https://arxiv.org/pdf/1711.05225.pdf
            loss = 0
            for i in range(len(neg_weights)):
                loss -= (neg_weights[i] * y_true[:, i] * K.log(y_pred[:, i] + epsilon) + 
                         pos_weights[i] * (1 - y_true[:, i]) * K.log(1 - y_pred[:, i] + epsilon))
            
            loss = K.sum(loss)
            return loss
        return weighted_loss
    
    model.compile(optimizer='adam', loss=get_weighted_loss(neg_weights, pos_weights))
    print("Compiled Model")

    model.load_weights("pretrained_model.h5")
    print("Loaded Weights")
    return model

The `load_C3M3_model()` function has been taken care of and as last time, its internals are out of the scope of this notebook.

In [0]:
# Load the model we used last time
model = load_C3M3_model()

Kindly recall from the previous notebook (GradCAM Part 1) that our model has 428 layers. 

We are now interested in getting the gradients when the model outputs a specific class. For this we will use Keras backend's `gradients(..)` function. This function requires two arguments: 

  - Loss (scalar tensor)
  - List of variables
  
Since we want the gradients with respect to the output, we can use our model's output tensor:

In [0]:
# Save model's output in a variable
y = model.output

# Print model's output
y

However this is not a scalar (aka rank-0) tensor because it has axes. To transform this tensor into a scalar we can slice it like this:

In [0]:
y = y[0]
y

It is still *not* a scalar tensor so we will have to slice it again:

In [0]:
y = y[0]
y

Now it is a scalar tensor!

The above slicing could be done in a single statement like this:

```python
y = y[0,0]
```

But the explicit version of it was shown for visibility purposes.

The first argument required by `gradients(..)` function is the loss, which we will like to get the gradient of, and the second is a list of parameters to compute the gradient with respect to. Since we are interested in getting the gradient of the output of the model with respect to the output of the last convolutional layer we need to specify the layer as we did  in the previous notebook:

In [0]:
# Save the desired layer in a variable
layer = model.get_layer("conv5_block16_concat")

# Compute gradient of model's output with respect to last conv layer's output
gradients = K.gradients(y, layer.output)

# Print gradients list
gradients

Notice that the gradients function returns a list of placeholder tensors. To get the actual placeholder we will get the first element of this list:

In [0]:
# Get first (and only) element in the list
gradients = gradients[0]

# Print tensor placeholder
gradients

As with the activations of the last convolutional layer in the previous notebook, we still need a function that uses this placeholder to compute the actual values for an input image. This can be done in the same manner as before. Remember this **function expects its arguments as lists or tuples**:

In [0]:
# Instantiate the function to compute the gradients
gradients_function = K.function([model.input], [gradients])

# Print the gradients function
gradients_function

Now that we have the function for computing the gradients, let's test it out on a particular image. Don't worry about the code to load the image, this has been taken care of for you, you should only care that an image ready to be processed will be saved in the x variable:

In [0]:
# Load dataframe that contains information about the dataset of images
df = pd.read_csv("train-small.csv")

# Path to the actual image
im_path = 'small_data/00000599_000.png'

# Load the image and save it to a variable
x = load_image(im_path, df, preprocess=False)

# Display the image
plt.imshow(x, cmap = 'gray')
plt.show()

We should normalize this image before going forward, this has also been taken care of:

In [0]:
# Calculate mean and standard deviation of a batch of images
mean, std = get_mean_std_per_batch(df)

# Normalize image
x = load_image_normalize(im_path, mean, std)

Now we have everything we need to compute the actual values of the gradients. In this case we should also **provide the input as a list or tuple**:

In [0]:
# Run the function on the image and save it in a variable
actual_gradients = gradients_function([x])

An important intermediary step is to trim the batch dimension which can be done like this:

In [0]:
# Remove batch dimension
actual_gradients = actual_gradients[0][0, :]

In [0]:
# Print shape of the gradients array
print(f"Gradients of model's output with respect to output of last convolutional layer have shape: {actual_gradients.shape}")

# Print gradients array
actual_gradients

Looks like everything worked out nicely! You will still have to wait for the assignment to see how these elements are used by Grad-CAM to get visual interpretations. Before you go you should know that there is a shortcut for these calculations by getting both elements from a single Keras function:

In [0]:
# Save multi-input Keras function in a variable
activations_and_gradients_function = K.function([model.input], [layer.output, gradients])

# Run the function on our image
act_x, grad_x = activations_and_gradients_function([x])

# Remove batch dimension for both arrays
act_x = act_x[0, :]
grad_x = grad_x[0, :]

In [0]:
# Print actual activations
print(act_x)

# Print actual gradients
print(grad_x)

**Congratulations on finishing this lecture notebook!** Hopefully you will now have a better understanding of how to leverage Keras's API power for computing gradients. Keep it up!